In [0]:
%sql
-- widgets
CREATE WIDGET TEXT catalog DEFAULT 'first_phase';
CREATE WIDGET TEXT silver_schema DEFAULT 'silver';

USE CATALOG ${catalog};
USE SCHEMA ${silver_schema};

-- check for NULLs in PK columns
WITH pk_cols AS (
  SELECT 'customer' tbl, 'c_custkey' col UNION ALL
  SELECT 'orders'  ,'o_orderkey'     UNION ALL
  SELECT 'lineitem','l_orderkey'     UNION ALL
  SELECT 'lineitem','l_linenumber'   UNION ALL
  SELECT 'nation'  ,'n_nationkey'    UNION ALL
  SELECT 'region'  ,'r_regionkey'    UNION ALL
  SELECT 'part'    ,'p_partkey'      UNION ALL
  SELECT 'partsupp','ps_partkey'     UNION ALL
  SELECT 'partsupp','ps_suppkey'     UNION ALL
  SELECT 'supplier','s_suppkey'
),
t AS (
  SELECT 'customer' tbl, COUNT_IF(c_custkey IS NULL) nulls FROM customer UNION ALL
  SELECT 'orders'  , COUNT_IF(o_orderkey IS NULL) FROM orders UNION ALL
  SELECT 'lineitem', COUNT_IF(l_orderkey IS NULL) FROM lineitem UNION ALL
  SELECT 'lineitem', COUNT_IF(l_linenumber IS NULL) FROM lineitem UNION ALL
  SELECT 'nation'  , COUNT_IF(n_nationkey IS NULL) FROM nation UNION ALL
  SELECT 'region'  , COUNT_IF(r_regionkey IS NULL) FROM region UNION ALL
  SELECT 'part'    , COUNT_IF(p_partkey IS NULL) FROM part UNION ALL
  SELECT 'partsupp', COUNT_IF(ps_partkey IS NULL) FROM partsupp UNION ALL
  SELECT 'partsupp', COUNT_IF(ps_suppkey IS NULL) FROM partsupp UNION ALL
  SELECT 'supplier', COUNT_IF(s_suppkey IS NULL) FROM supplier
)
SELECT * FROM t ORDER BY tbl;


In [0]:
%sql
-- set NOT NULL on all PK columns (fails if any NULLs are present)
ALTER TABLE customer  ALTER COLUMN c_custkey     SET NOT NULL;
ALTER TABLE orders    ALTER COLUMN o_orderkey    SET NOT NULL;
ALTER TABLE lineitem  ALTER COLUMN l_orderkey    SET NOT NULL;
ALTER TABLE lineitem  ALTER COLUMN l_linenumber  SET NOT NULL;
ALTER TABLE nation    ALTER COLUMN n_nationkey   SET NOT NULL;
ALTER TABLE region    ALTER COLUMN r_regionkey   SET NOT NULL;
ALTER TABLE part      ALTER COLUMN p_partkey     SET NOT NULL;
ALTER TABLE partsupp  ALTER COLUMN ps_partkey    SET NOT NULL;
ALTER TABLE partsupp  ALTER COLUMN ps_suppkey    SET NOT NULL;
ALTER TABLE supplier  ALTER COLUMN s_suppkey     SET NOT NULL;


In [0]:
%sql
-- drop if you retried earlier
ALTER TABLE customer  DROP CONSTRAINT IF EXISTS pk_customer;
ALTER TABLE orders    DROP CONSTRAINT IF EXISTS pk_orders;
ALTER TABLE lineitem  DROP CONSTRAINT IF EXISTS pk_lineitem;
ALTER TABLE nation    DROP CONSTRAINT IF EXISTS pk_nation;
ALTER TABLE region    DROP CONSTRAINT IF EXISTS pk_region;
ALTER TABLE part      DROP CONSTRAINT IF EXISTS pk_part;
ALTER TABLE partsupp  DROP CONSTRAINT IF EXISTS pk_partsupp;
ALTER TABLE supplier  DROP CONSTRAINT IF EXISTS pk_supplier;

-- add PKs
ALTER TABLE customer  ADD CONSTRAINT pk_customer  PRIMARY KEY (c_custkey) NOT ENFORCED;
ALTER TABLE orders    ADD CONSTRAINT pk_orders    PRIMARY KEY (o_orderkey) NOT ENFORCED;
ALTER TABLE lineitem  ADD CONSTRAINT pk_lineitem  PRIMARY KEY (l_orderkey, l_linenumber) NOT ENFORCED;
ALTER TABLE nation    ADD CONSTRAINT pk_nation    PRIMARY KEY (n_nationkey) NOT ENFORCED;
ALTER TABLE region    ADD CONSTRAINT pk_region    PRIMARY KEY (r_regionkey) NOT ENFORCED;
ALTER TABLE part      ADD CONSTRAINT pk_part      PRIMARY KEY (p_partkey) NOT ENFORCED;
ALTER TABLE partsupp  ADD CONSTRAINT pk_partsupp  PRIMARY KEY (ps_partkey, ps_suppkey) NOT ENFORCED;
ALTER TABLE supplier  ADD CONSTRAINT pk_supplier  PRIMARY KEY (s_suppkey) NOT ENFORCED;


In [0]:
%sql
ALTER TABLE customer  ADD CONSTRAINT fk_customer_nation
  FOREIGN KEY (c_nationkey) REFERENCES nation(n_nationkey) NOT ENFORCED;

ALTER TABLE orders    ADD CONSTRAINT fk_orders_customer
  FOREIGN KEY (o_custkey) REFERENCES customer(c_custkey) NOT ENFORCED;

ALTER TABLE lineitem  ADD CONSTRAINT fk_lineitem_orders
  FOREIGN KEY (l_orderkey) REFERENCES orders(o_orderkey) NOT ENFORCED;
ALTER TABLE lineitem  ADD CONSTRAINT fk_lineitem_part
  FOREIGN KEY (l_partkey) REFERENCES part(p_partkey) NOT ENFORCED;
ALTER TABLE lineitem  ADD CONSTRAINT fk_lineitem_supplier
  FOREIGN KEY (l_suppkey) REFERENCES supplier(s_suppkey) NOT ENFORCED;

ALTER TABLE supplier  ADD CONSTRAINT fk_supplier_nation
  FOREIGN KEY (s_nationkey) REFERENCES nation(n_nationkey) NOT ENFORCED;

ALTER TABLE nation    ADD CONSTRAINT fk_nation_region
  FOREIGN KEY (n_regionkey) REFERENCES region(r_regionkey) NOT ENFORCED;

ALTER TABLE partsupp  ADD CONSTRAINT fk_partsupp_part
  FOREIGN KEY (ps_partkey) REFERENCES part(p_partkey) NOT ENFORCED;
ALTER TABLE partsupp  ADD CONSTRAINT fk_partsupp_supplier
  FOREIGN KEY (ps_suppkey) REFERENCES supplier(s_suppkey) NOT ENFORCED;


In [0]:
%sql
-- if widgets weren’t created in this *same* notebook, create them now
CREATE WIDGET TEXT catalog DEFAULT 'main';
CREATE WIDGET TEXT bronze_schema DEFAULT 'bronze';
CREATE WIDGET TEXT silver_schema DEFAULT 'silver';

-- see what your session thinks is current
SELECT current_catalog(), current_schema();

-- list catalogs you can use
SHOW CATALOGS;

-- switch to a real catalog you have (often 'main' or 'hive_metastore')
USE CATALOG ${catalog};

-- create/use schemas if needed
CREATE SCHEMA IF NOT EXISTS ${bronze_schema};
CREATE SCHEMA IF NOT EXISTS ${silver_schema};
USE SCHEMA ${silver_schema};


In [0]:
%sql
-- widgets
CREATE WIDGET TEXT catalog DEFAULT 'main';
CREATE WIDGET TEXT bronze_schema DEFAULT 'bronze';
CREATE WIDGET TEXT silver_schema DEFAULT 'silver';

USE CATALOG ${catalog};

-- 0) convenience views
-- 1) Explicit row counts (bronze vs silver)
CREATE OR REPLACE TEMP VIEW _bronze_counts AS
SELECT 'customer' AS tbl, (SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.customer)  AS n UNION ALL
SELECT 'orders'   , (SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.orders)         UNION ALL
SELECT 'lineitem' , (SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.lineitem)       UNION ALL
SELECT 'nation'   , (SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.nation)         UNION ALL
SELECT 'region'   , (SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.region)         UNION ALL
SELECT 'part'     , (SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.part)           UNION ALL
SELECT 'partsupp' , (SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.partsupp)       UNION ALL
SELECT 'supplier' , (SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.supplier);

CREATE OR REPLACE TEMP VIEW _silver_counts AS
SELECT 'customer' AS tbl, (SELECT COUNT(*) FROM ${catalog}.${silver_schema}.customer)  AS n UNION ALL
SELECT 'orders'   , (SELECT COUNT(*) FROM ${catalog}.${silver_schema}.orders)         UNION ALL
SELECT 'lineitem' , (SELECT COUNT(*) FROM ${catalog}.${silver_schema}.lineitem)       UNION ALL
SELECT 'nation'   , (SELECT COUNT(*) FROM ${catalog}.${silver_schema}.nation)         UNION ALL
SELECT 'region'   , (SELECT COUNT(*) FROM ${catalog}.${silver_schema}.region)         UNION ALL
SELECT 'part'     , (SELECT COUNT(*) FROM ${catalog}.${silver_schema}.part)           UNION ALL
SELECT 'partsupp' , (SELECT COUNT(*) FROM ${catalog}.${silver_schema}.partsupp)       UNION ALL
SELECT 'supplier' , (SELECT COUNT(*) FROM ${catalog}.${silver_schema}.supplier);

-- comparison table
SELECT 'row_count_match' AS test,
       b.tbl AS table_name,
       b.n   AS bronze_rows,
       s.n   AS silver_rows,
       CASE WHEN b.n = s.n THEN 'PASS' ELSE 'FAIL' END AS status
FROM _bronze_counts b
JOIN _silver_counts s USING (tbl)
ORDER BY table_name;


-- 2) schema diff (missing/extra/type mismatch)
-- missing in silver
SELECT 'schema_missing_in_silver' AS test, s.table_name, s.column_name, s.data_type AS src_type,
       NULL AS silver_type, 'FAIL' AS status
FROM ${catalog}.information_schema.columns s
LEFT JOIN ${catalog}.information_schema.columns t
  ON t.table_schema='${silver_schema}' AND t.table_name=s.table_name AND t.column_name=s.column_name
WHERE s.table_schema='${bronze_schema}' AND t.column_name IS NULL
ORDER BY table_name, column_name;

-- extra in silver
SELECT 'schema_extra_in_silver' AS test, t.table_name, t.column_name, NULL AS src_type,
       t.data_type AS silver_type, 'FAIL' AS status
FROM ${catalog}.information_schema.columns t
LEFT JOIN ${catalog}.information_schema.columns s
  ON s.table_schema='${bronze_schema}' AND s.table_name=t.table_name AND s.column_name=t.column_name
WHERE t.table_schema='${silver_schema}' AND s.column_name IS NULL
ORDER BY table_name, column_name;

-- type mismatch
SELECT 'schema_type_mismatch' AS test, s.table_name, s.column_name, s.data_type AS src_type,
       t.data_type AS silver_type, CASE WHEN s.data_type=t.data_type THEN 'PASS' ELSE 'FAIL' END AS status
FROM ${catalog}.information_schema.columns s
JOIN ${catalog}.information_schema.columns t
  ON t.table_schema='${silver_schema}' AND t.table_name=s.table_name AND t.column_name=s.column_name
WHERE s.table_schema='${bronze_schema}' AND s.data_type<>t.data_type
ORDER BY table_name, column_name;

-- 3) PK nulls & duplicates = 0 expected
WITH pk_nulls AS (
  SELECT 'customer' tbl, COUNT_IF(c_custkey IS NULL) n FROM ${catalog}.${silver_schema}.customer UNION ALL
  SELECT 'orders'  , COUNT_IF(o_orderkey IS NULL)   FROM ${catalog}.${silver_schema}.orders   UNION ALL
  SELECT 'lineitem', COUNT_IF(l_orderkey IS NULL OR l_linenumber IS NULL)
                                              FROM ${catalog}.${silver_schema}.lineitem UNION ALL
  SELECT 'nation'  , COUNT_IF(n_nationkey IS NULL)  FROM ${catalog}.${silver_schema}.nation  UNION ALL
  SELECT 'region'  , COUNT_IF(r_regionkey IS NULL)  FROM ${catalog}.${silver_schema}.region  UNION ALL
  SELECT 'part'    , COUNT_IF(p_partkey IS NULL)    FROM ${catalog}.${silver_schema}.part    UNION ALL
  SELECT 'partsupp', COUNT_IF(ps_partkey IS NULL OR ps_suppkey IS NULL)
                                              FROM ${catalog}.${silver_schema}.partsupp UNION ALL
  SELECT 'supplier', COUNT_IF(s_suppkey IS NULL)    FROM ${catalog}.${silver_schema}.supplier
),
pk_dupes AS (
  SELECT 'customer' tbl, COUNT(*) n FROM (
    SELECT c_custkey, COUNT(*) c FROM ${catalog}.${silver_schema}.customer GROUP BY c_custkey HAVING COUNT(*)>1
  ) UNION ALL
  SELECT 'orders', COUNT(*) FROM (
    SELECT o_orderkey, COUNT(*) c FROM ${catalog}.${silver_schema}.orders GROUP BY o_orderkey HAVING COUNT(*)>1
  ) UNION ALL
  SELECT 'lineitem', COUNT(*) FROM (
    SELECT l_orderkey, l_linenumber, COUNT(*) c
    FROM ${catalog}.${silver_schema}.lineitem GROUP BY l_orderkey,l_linenumber HAVING COUNT(*)>1
  ) UNION ALL
  SELECT 'nation', COUNT(*) FROM (
    SELECT n_nationkey, COUNT(*) c FROM ${catalog}.${silver_schema}.nation GROUP BY n_nationkey HAVING COUNT(*)>1
  ) UNION ALL
  SELECT 'region', COUNT(*) FROM (
    SELECT r_regionkey, COUNT(*) c FROM ${catalog}.${silver_schema}.region GROUP BY r_regionkey HAVING COUNT(*)>1
  ) UNION ALL
  SELECT 'part', COUNT(*) FROM (
    SELECT p_partkey, COUNT(*) c FROM ${catalog}.${silver_schema}.part GROUP BY p_partkey HAVING COUNT(*)>1
  ) UNION ALL
  SELECT 'supplier', COUNT(*) FROM (
    SELECT s_suppkey, COUNT(*) c FROM ${catalog}.${silver_schema}.supplier GROUP BY s_suppkey HAVING COUNT(*)>1
  ) UNION ALL
  SELECT 'partsupp', COUNT(*) FROM (
    SELECT ps_partkey, ps_suppkey, COUNT(*) c
    FROM ${catalog}.${silver_schema}.partsupp GROUP BY ps_partkey,ps_suppkey HAVING COUNT(*)>1
  )
)
SELECT 'pk_nulls' AS test, n.tbl AS table_name, n.n AS issues,
       CASE WHEN n.n=0 THEN 'PASS' ELSE 'FAIL' END AS status
FROM pk_nulls n
UNION ALL
SELECT 'pk_duplicates', d.tbl, d.n,
       CASE WHEN d.n=0 THEN 'PASS' ELSE 'FAIL' END
FROM pk_dupes d
ORDER BY table_name, test;

-- 4) FK orphan checks = 0 expected
WITH orphans AS (
  SELECT 'orders.o_custkey -> customer' rel, COUNT(*) n
  FROM ${catalog}.${silver_schema}.orders o
  LEFT ANTI JOIN ${catalog}.${silver_schema}.customer c ON o.o_custkey=c.c_custkey
  UNION ALL
  SELECT 'customer.c_nationkey -> nation', COUNT(*)
  FROM ${catalog}.${silver_schema}.customer c
  LEFT ANTI JOIN ${catalog}.${silver_schema}.nation n ON c.c_nationkey=n.n_nationkey
  UNION ALL
  SELECT 'supplier.s_nationkey -> nation', COUNT(*)
  FROM ${catalog}.${silver_schema}.supplier s
  LEFT ANTI JOIN ${catalog}.${silver_schema}.nation n ON s.s_nationkey=n.n_nationkey
  UNION ALL
  SELECT 'nation.n_regionkey -> region', COUNT(*)
  FROM ${catalog}.${silver_schema}.nation n
  LEFT ANTI JOIN ${catalog}.${silver_schema}.region r ON n.n_regionkey=r.r_regionkey
  UNION ALL
  SELECT 'lineitem.l_orderkey -> orders', COUNT(*)
  FROM ${catalog}.${silver_schema}.lineitem l
  LEFT ANTI JOIN ${catalog}.${silver_schema}.orders o ON l.l_orderkey=o.o_orderkey
  UNION ALL
  SELECT 'lineitem.l_partkey -> part', COUNT(*)
  FROM ${catalog}.${silver_schema}.lineitem l
  LEFT ANTI JOIN ${catalog}.${silver_schema}.part p ON l.l_partkey=p.p_partkey
  UNION ALL
  SELECT 'lineitem.l_suppkey -> supplier', COUNT(*)
  FROM ${catalog}.${silver_schema}.lineitem l
  LEFT ANTI JOIN ${catalog}.${silver_schema}.supplier s ON l.l_suppkey=s.s_suppkey
  UNION ALL
  SELECT 'partsupp.ps_partkey -> part', COUNT(*)
  FROM ${catalog}.${silver_schema}.partsupp ps
  LEFT ANTI JOIN ${catalog}.${silver_schema}.part p ON ps.ps_partkey=p.p_partkey
  UNION ALL
  SELECT 'partsupp.ps_suppkey -> supplier', COUNT(*)
  FROM ${catalog}.${silver_schema}.partsupp ps
  LEFT ANTI JOIN ${catalog}.${silver_schema}.supplier s ON ps.ps_suppkey=s.s_suppkey
)
SELECT 'fk_orphans' AS test, rel AS relation, n AS issues,
       CASE WHEN n=0 THEN 'PASS' ELSE 'FAIL' END AS status
FROM orphans
ORDER BY relation;

-- 5) key set parity (bronze vs silver) = 0 diffs expected
WITH keydiffs AS (
  -- customer
  SELECT 'customer' tbl, 'silver_missing_keys' kind, COUNT(*) n
  FROM ${catalog}.${bronze_schema}.customer b
  LEFT ANTI JOIN ${catalog}.${silver_schema}.customer s ON b.c_custkey=s.c_custkey
  UNION ALL
  SELECT 'customer','silver_extra_keys', COUNT(*)
  FROM ${catalog}.${silver_schema}.customer s
  LEFT ANTI JOIN ${catalog}.${bronze_schema}.customer b ON s.c_custkey=b.c_custkey

  UNION ALL
  -- orders
  SELECT 'orders','silver_missing_keys', COUNT(*)
  FROM ${catalog}.${bronze_schema}.orders b
  LEFT ANTI JOIN ${catalog}.${silver_schema}.orders s ON b.o_orderkey=s.o_orderkey
  UNION ALL
  SELECT 'orders','silver_extra_keys', COUNT(*)
  FROM ${catalog}.${silver_schema}.orders s
  LEFT ANTI JOIN ${catalog}.${bronze_schema}.orders b ON s.o_orderkey=b.o_orderkey

  UNION ALL
  -- lineitem composite
  SELECT 'lineitem','silver_missing_keys', COUNT(*)
  FROM ${catalog}.${bronze_schema}.lineitem b
  LEFT ANTI JOIN ${catalog}.${silver_schema}.lineitem s
    ON b.l_orderkey=s.l_orderkey AND b.l_linenumber=s.l_linenumber
  UNION ALL
  SELECT 'lineitem','silver_extra_keys', COUNT(*)
  FROM ${catalog}.${silver_schema}.lineitem s
  LEFT ANTI JOIN ${catalog}.${bronze_schema}.lineitem b
    ON s.l_orderkey=b.l_orderkey AND s.l_linenumber=b.l_linenumber

  UNION ALL
  -- nation
  SELECT 'nation','silver_missing_keys', COUNT(*)
  FROM ${catalog}.${bronze_schema}.nation b
  LEFT ANTI JOIN ${catalog}.${silver_schema}.nation s ON b.n_nationkey=s.n_nationkey
  UNION ALL
  SELECT 'nation','silver_extra_keys', COUNT(*)
  FROM ${catalog}.${silver_schema}.nation s
  LEFT ANTI JOIN ${catalog}.${bronze_schema}.nation b ON s.n_nationkey=b.n_nationkey

  UNION ALL
  -- region
  SELECT 'region','silver_missing_keys', COUNT(*)
  FROM ${catalog}.${bronze_schema}.region b
  LEFT ANTI JOIN ${catalog}.${silver_schema}.region s ON b.r_regionkey=s.r_regionkey
  UNION ALL
  SELECT 'region','silver_extra_keys', COUNT(*)
  FROM ${catalog}.${silver_schema}.region s
  LEFT ANTI JOIN ${catalog}.${bronze_schema}.region b ON s.r_regionkey=b.r_regionkey

  UNION ALL
  -- part
  SELECT 'part','silver_missing_keys', COUNT(*)
  FROM ${catalog}.${bronze_schema}.part b
  LEFT ANTI JOIN ${catalog}.${silver_schema}.part s ON b.p_partkey=s.p_partkey
  UNION ALL
  SELECT 'part','silver_extra_keys', COUNT(*)
  FROM ${catalog}.${silver_schema}.part s
  LEFT ANTI JOIN ${catalog}.${bronze_schema}.part b ON s.p_partkey=b.p_partkey

  UNION ALL
  -- supplier
  SELECT 'supplier','silver_missing_keys', COUNT(*)
  FROM ${catalog}.${bronze_schema}.supplier b
  LEFT ANTI JOIN ${catalog}.${silver_schema}.supplier s ON b.s_suppkey=s.s_suppkey
  UNION ALL
  SELECT 'supplier','silver_extra_keys', COUNT(*)
  FROM ${catalog}.${silver_schema}.supplier s
  LEFT ANTI JOIN ${catalog}.${bronze_schema}.supplier b ON s.s_suppkey=b.s_suppkey

  UNION ALL
  -- partsupp composite
  SELECT 'partsupp','silver_missing_keys', COUNT(*)
  FROM ${catalog}.${bronze_schema}.partsupp b
  LEFT ANTI JOIN ${catalog}.${silver_schema}.partsupp s
    ON b.ps_partkey=s.ps_partkey AND b.ps_suppkey=s.ps_suppkey
  UNION ALL
  SELECT 'partsupp','silver_extra_keys', COUNT(*)
  FROM ${catalog}.${silver_schema}.partsupp s
  LEFT ANTI JOIN ${catalog}.${bronze_schema}.partsupp b
    ON s.ps_partkey=b.ps_partkey AND s.ps_suppkey=b.ps_suppkey
)
SELECT 'key_set_parity' AS test, tbl AS table_name, kind, n AS issues,
       CASE WHEN n=0 THEN 'PASS' ELSE 'FAIL' END AS status
FROM keydiffs
ORDER BY table_name, kind;

-- 6) FINAL SUMMARY (one row per check category)
WITH a AS (
  SELECT CASE WHEN MIN(CASE WHEN b.n = s.n THEN 1 ELSE 0 END) = 1
              THEN 'PASS' ELSE 'FAIL' END AS status
  FROM _bronze_counts b
  JOIN _silver_counts s USING (tbl)
),
b AS (
  SELECT CASE WHEN COUNT(*)=0 THEN 'PASS' ELSE 'FAIL' END AS status
  FROM (
    SELECT 1
    FROM ${catalog}.information_schema.columns s
    LEFT JOIN ${catalog}.information_schema.columns t
      ON t.table_schema='${silver_schema}' AND t.table_name=s.table_name AND t.column_name=s.column_name
    WHERE s.table_schema='${bronze_schema}' AND t.column_name IS NULL
    UNION ALL
    SELECT 1
    FROM ${catalog}.information_schema.columns t
    LEFT JOIN ${catalog}.information_schema.columns s
      ON s.table_schema='${bronze_schema}' AND s.table_name=t.table_name AND s.column_name=t.column_name
    WHERE t.table_schema='${silver_schema}' AND s.column_name IS NULL
    UNION ALL
    SELECT 1
    FROM ${catalog}.information_schema.columns s
    JOIN ${catalog}.information_schema.columns t
      ON t.table_schema='${silver_schema}' AND t.table_name=s.table_name AND t.column_name=s.column_name
    WHERE s.table_schema='${bronze_schema}' AND s.data_type<>t.data_type
  )
),
c AS (
  SELECT CASE WHEN SUM(n)=0 THEN 'PASS' ELSE 'FAIL' END AS status
  FROM (
    WITH pk_nulls AS (
      SELECT COUNT_IF(c_custkey IS NULL) n FROM ${catalog}.${silver_schema}.customer UNION ALL
      SELECT COUNT_IF(o_orderkey IS NULL) FROM ${catalog}.${silver_schema}.orders UNION ALL
      SELECT COUNT_IF(l_orderkey IS NULL OR l_linenumber IS NULL) FROM ${catalog}.${silver_schema}.lineitem UNION ALL
      SELECT COUNT_IF(n_nationkey IS NULL) FROM ${catalog}.${silver_schema}.nation UNION ALL
      SELECT COUNT_IF(r_regionkey IS NULL) FROM ${catalog}.${silver_schema}.region UNION ALL
      SELECT COUNT_IF(p_partkey IS NULL) FROM ${catalog}.${silver_schema}.part UNION ALL
      SELECT COUNT_IF(ps_partkey IS NULL OR ps_suppkey IS NULL) FROM ${catalog}.${silver_schema}.partsupp UNION ALL
      SELECT COUNT_IF(s_suppkey IS NULL) FROM ${catalog}.${silver_schema}.supplier
    ),
    pk_dupes AS (
      SELECT COUNT(*) n FROM (SELECT c_custkey, COUNT(*) c FROM ${catalog}.${silver_schema}.customer GROUP BY c_custkey HAVING COUNT(*)>1)
      UNION ALL SELECT COUNT(*) FROM (SELECT o_orderkey, COUNT(*) c FROM ${catalog}.${silver_schema}.orders GROUP BY o_orderkey HAVING COUNT(*)>1)
      UNION ALL SELECT COUNT(*) FROM (SELECT l_orderkey,l_linenumber, COUNT(*) c FROM ${catalog}.${silver_schema}.lineitem GROUP BY l_orderkey,l_linenumber HAVING COUNT(*)>1)
      UNION ALL SELECT COUNT(*) FROM (SELECT n_nationkey, COUNT(*) c FROM ${catalog}.${silver_schema}.nation GROUP BY n_nationkey HAVING COUNT(*)>1)
      UNION ALL SELECT COUNT(*) FROM (SELECT r_regionkey, COUNT(*) c FROM ${catalog}.${silver_schema}.region GROUP BY r_regionkey HAVING COUNT(*)>1)
      UNION ALL SELECT COUNT(*) FROM (SELECT p_partkey, COUNT(*) c FROM ${catalog}.${silver_schema}.part GROUP BY p_partkey HAVING COUNT(*)>1)
      UNION ALL SELECT COUNT(*) FROM (SELECT s_suppkey, COUNT(*) c FROM ${catalog}.${silver_schema}.supplier GROUP BY s_suppkey HAVING COUNT(*)>1)
      UNION ALL SELECT COUNT(*) FROM (SELECT ps_partkey,ps_suppkey, COUNT(*) c FROM ${catalog}.${silver_schema}.partsupp GROUP BY ps_partkey,ps_suppkey HAVING COUNT(*)>1)
    )
    SELECT * FROM pk_nulls UNION ALL SELECT * FROM pk_dupes
  )
),
d AS (
  SELECT CASE WHEN SUM(n)=0 THEN 'PASS' ELSE 'FAIL' END AS status
  FROM (
    SELECT COUNT(*) n FROM ${catalog}.${silver_schema}.orders  o LEFT ANTI JOIN ${catalog}.${silver_schema}.customer c ON o.o_custkey=c.c_custkey
    UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.customer c LEFT ANTI JOIN ${catalog}.${silver_schema}.nation n ON c.c_nationkey=n.n_nationkey
    UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.supplier s LEFT ANTI JOIN ${catalog}.${silver_schema}.nation n ON s.s_nationkey=n.n_nationkey
    UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.nation   n LEFT ANTI JOIN ${catalog}.${silver_schema}.region r ON n.n_regionkey=r.r_regionkey
    UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.lineitem l LEFT ANTI JOIN ${catalog}.${silver_schema}.orders   o ON l.l_orderkey=o.o_orderkey
    UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.lineitem l LEFT ANTI JOIN ${catalog}.${silver_schema}.part     p ON l.l_partkey=p.p_partkey
    UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.lineitem l LEFT ANTI JOIN ${catalog}.${silver_schema}.supplier s ON l.l_suppkey=s.s_suppkey
    UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.partsupp ps LEFT ANTI JOIN ${catalog}.${silver_schema}.part     p ON ps.ps_partkey=p.p_partkey
    UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.partsupp ps LEFT ANTI JOIN ${catalog}.${silver_schema}.supplier s ON ps.ps_suppkey=s.s_suppkey
  )
),
e AS (
  SELECT CASE WHEN SUM(n)=0 THEN 'PASS' ELSE 'FAIL' END AS status
  FROM (
    WITH kd AS (
      SELECT COUNT(*) n FROM ${catalog}.${bronze_schema}.customer b LEFT ANTI JOIN ${catalog}.${silver_schema}.customer s ON b.c_custkey=s.c_custkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.customer s LEFT ANTI JOIN ${catalog}.${bronze_schema}.customer b ON s.c_custkey=b.c_custkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.orders b LEFT ANTI JOIN ${catalog}.${silver_schema}.orders s ON b.o_orderkey=s.o_orderkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.orders s LEFT ANTI JOIN ${catalog}.${bronze_schema}.orders b ON s.o_orderkey=b.o_orderkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.lineitem b LEFT ANTI JOIN ${catalog}.${silver_schema}.lineitem s ON b.l_orderkey=s.l_orderkey AND b.l_linenumber=s.l_linenumber
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.lineitem s LEFT ANTI JOIN ${catalog}.${bronze_schema}.lineitem b ON s.l_orderkey=b.l_orderkey AND s.l_linenumber=b.l_linenumber
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.nation b LEFT ANTI JOIN ${catalog}.${silver_schema}.nation s ON b.n_nationkey=s.n_nationkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.nation s LEFT ANTI JOIN ${catalog}.${bronze_schema}.nation b ON s.n_nationkey=b.n_nationkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.region b LEFT ANTI JOIN ${catalog}.${silver_schema}.region s ON b.r_regionkey=s.r_regionkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.region s LEFT ANTI JOIN ${catalog}.${bronze_schema}.region b ON s.r_regionkey=b.r_regionkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.part b LEFT ANTI JOIN ${catalog}.${silver_schema}.part s ON b.p_partkey=s.p_partkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.part s LEFT ANTI JOIN ${catalog}.${bronze_schema}.part b ON s.p_partkey=b.p_partkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.supplier b LEFT ANTI JOIN ${catalog}.${silver_schema}.supplier s ON b.s_suppkey=s.s_suppkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.supplier s LEFT ANTI JOIN ${catalog}.${bronze_schema}.supplier b ON s.s_suppkey=b.s_suppkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${bronze_schema}.partsupp b LEFT ANTI JOIN ${catalog}.${silver_schema}.partsupp s ON b.ps_partkey=s.ps_partkey AND b.ps_suppkey=s.ps_suppkey
      UNION ALL SELECT COUNT(*) FROM ${catalog}.${silver_schema}.partsupp s LEFT ANTI JOIN ${catalog}.${bronze_schema}.partsupp b ON s.ps_partkey=b.ps_partkey AND s.ps_suppkey=b.ps_suppkey
    )
    SELECT * FROM kd
  )
)
SELECT 'SUMMARY' section,
       'row_counts' AS check_name, a.status FROM a
UNION ALL
SELECT 'SUMMARY','schema_alignment', b.status FROM b
UNION ALL
SELECT 'SUMMARY','pk_health', c.status FROM c
UNION ALL
SELECT 'SUMMARY','fk_health', d.status FROM d
UNION ALL
SELECT 'SUMMARY','key_set_parity', e.status FROM e;
